In [1]:
import pandas as pd
import numpy as np
import os
import cv2
#import tensorflow as tf
import glob
import h5py
import idx2numpy as idx
import warnings
import re
from PIL import Image
from tqdm import tqdm
# from progress.bar import IncrementalBar

# from data_generator.object_detection_2d_data_generator import DataGenerator
# import xml.etree.ElementTree as ET

In [2]:
warnings.filterwarnings('ignore')

#входное разрешение модели
IMAGE_HEIGHT = 28 # Height of the input images
IMAGE_WIDTH = 28 #Width of the input images
WORK_DIR = 'D:\work\\test_comp_vision\datasets\!_lines_w25_parsed_full' #путь к папке с файлами
# DATASET_H5 = 'D:\work\\test_comp_vision\datasets\!_lines_w25_dataset.h5'
DATASET_IDX_IMG = r'D:\work\test_comp_vision\datasets\!_lines_w25_dataset_images_100k_upper2.idx'
DATASET_IDX_CLS = r'D:\work\test_comp_vision\datasets\!_lines_w25_dataset_classes_100k_upper2.idx'
DATASET_SIZE = 100000      # какую часть изображений используем для создания датасета. 0 - если нужны все данные
# LABELS = '0123456789АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдеёжзийклмнопрстуфхцчшщъыьэюя'
LABELS = '12456789АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ'   # for only uppercase letters without '0' and '3' numbers
EXPORT_NAME_CSV = '!img_classes_upper2.csv'

In [6]:
"""
Структура папок:
base_folder
    image_0
        image_0_0.jpg
        image_0_0.txt
        image_0_1.jpg
        image_0_1.txt
        ...
    image_1
        image_1_0.jpg
        image_1_0.txt
        image_1_1.jpg
        image_1_1.txt
        ...
        
"""
# TODO заменить работу с индексами над путями к файлам на нормальную обработку разрешения файлов
def get_dataframe(path):
    # pattern = r'/^(.+)(\.[^ .]+)?$/'      # для удаления разрешения из пути к файлу
    
    symbol_classes = []
    for folder in tqdm(os.listdir(path)):
        # пропускаем папки с буквой 'а', т.к. там версия текста в нижнем регистре
        if folder[:-1] == 'a' : continue     # В целом эту проверку можно не делать, т.к. в любом случаем пропускаем буквы не из списка LABEL
        # TODO добавить ограничение на количество итераций здесь, чтобы не создавать DataSet больше нужного
        for img_file in glob.glob(f'{path}/{folder}/*.jpg'):
            txt_file =  f"{img_file[:-3]}txt"    # меняем разрешение для открытия текста [:-3] TODO заменить на pathlib.stem
            # print(f'IMG_file: {img_file}\nTXT_file: {txt_file}')
            with open(txt_file, 'r') as f:
                symbol_class = f.read()
                # symbol_class = f.read().upper()      # читаем обозначение буквы и приводим ее к верхнему регистру
                if symbol_class not in LABELS: continue      # пропускаем все символы, не входящие в искомое множество
                
            # TODO - проверить без этого преобразования но с использоованием в дальнейшем строки с ведущей r'string'
            # или же переписать весь код на использование Pathlib вместо строк с адресом в win-формате
            value = (img_file.replace('\\', '/'), symbol_class)  # меняем системный флеш Винды на тот, что понимает Python
            symbol_classes.append(value)
    column_names = ['file_name', 'class']
    return pd.DataFrame(symbol_classes, columns=column_names)

In [7]:
#Создаем DataFrame в котором хранится путь к фалу/изображению и его класс
base_path = WORK_DIR #путь к основной папке
df_text = pd.DataFrame()

if DATASET_SIZE > 0:      # если ограничение задано, то используем его при выборке данных
    df_text = df_text.append(get_dataframe(base_path)[:DATASET_SIZE])
else:
    df_text = df_text.append(get_dataframe(base_path))
df_text

100%|███████████████████████████████████████████████████████████████████████████| 20172/20172 [00:46<00:00, 435.45it/s]


,file_name,class
0,D:/work/test_comp_vision/datasets/!_lines_w25_...,В
1,D:/work/test_comp_vision/datasets/!_lines_w25_...,Е
2,D:/work/test_comp_vision/datasets/!_lines_w25_...,Е
3,D:/work/test_comp_vision/datasets/!_lines_w25_...,Р
4,D:/work/test_comp_vision/datasets/!_lines_w25_...,А
...,...,...
99995,D:/work/test_comp_vision/datasets/!_lines_w25_...,О
99996,D:/work/test_comp_vision/datasets/!_lines_w25_...,В
99997,D:/work/test_comp_vision/datasets/!_lines_w25_...,С
99998,D:/work/test_comp_vision/datasets/!_lines_w25_...,Е


In [8]:
# Преобразуем классы к числовым значениям
labels_to_int = {}
for i, label in enumerate(LABELS):
    labels_to_int[label] = i
    # labels_to_int[i] = label
print(labels_to_int)

df_images = pd.DataFrame({'image_path':df_text['file_name'],
                   'class_id':[labels_to_int[label] for label in df_text['class']]})
# df_images.to_csv(EXPORT_NAME_CSV, index=False)
df_images

{'1': 0, '2': 1, '4': 2, '5': 3, '6': 4, '7': 5, '8': 6, '9': 7, 'А': 8, 'Б': 9, 'В': 10, 'Г': 11, 'Д': 12, 'Е': 13, 'Ё': 14, 'Ж': 15, 'З': 16, 'И': 17, 'Й': 18, 'К': 19, 'Л': 20, 'М': 21, 'Н': 22, 'О': 23, 'П': 24, 'Р': 25, 'С': 26, 'Т': 27, 'У': 28, 'Ф': 29, 'Х': 30, 'Ц': 31, 'Ч': 32, 'Ш': 33, 'Щ': 34, 'Ъ': 35, 'Ы': 36, 'Ь': 37, 'Э': 38, 'Ю': 39, 'Я': 40}


,image_path,class_id
0,D:/work/test_comp_vision/datasets/!_lines_w25_...,10
1,D:/work/test_comp_vision/datasets/!_lines_w25_...,13
2,D:/work/test_comp_vision/datasets/!_lines_w25_...,13
3,D:/work/test_comp_vision/datasets/!_lines_w25_...,25
4,D:/work/test_comp_vision/datasets/!_lines_w25_...,8
...,...,...
99995,D:/work/test_comp_vision/datasets/!_lines_w25_...,23
99996,D:/work/test_comp_vision/datasets/!_lines_w25_...,10
99997,D:/work/test_comp_vision/datasets/!_lines_w25_...,26
99998,D:/work/test_comp_vision/datasets/!_lines_w25_...,13


In [13]:
row = 134
print(df_images.iloc[row]['image_path'])
print(df_images.iloc[row]['class_id'])

D:/work/test_comp_vision/datasets/!_lines_w25_parsed_full/10004_b/10004_b_20.jpg
13


In [14]:
print(df_images.shape)
for i, row in df_images[:5].iterrows():  
    print(row['class_id'])

(100000, 2)
10
13
13
25
8


## .IDX файл генератор

In [15]:
with Image.open(r"D:\work\test_comp_vision\test_for_MindSet\pass_temp\0\0\0-0.jpg") as image:
        print(image.size)
        parsed_image = np.asarray(image)
        print(parsed_image.shape)
        parsed_image2 = np.asarray([parsed_image])
        print(parsed_image2.shape)
        

(28, 28)
(28, 28)
(1, 28, 28)


In [16]:
# преобразуем все изображения в массив np.array()

parsed_images = []              # создаем два файла: в одном хранятся изображения в формате np.array
parsed_classes = []             # во втором хранятся классы
print(f"Общее количество: {df_images.shape[0]}")

for i, row in tqdm(df_images.iterrows()):
    
    with Image.open(row['image_path']) as image:
        parsed_images.append(np.asarray(image))  #открываем картинку по ссылке, преобразуем в массив
    #print(parsed_images[row[0]].shape)
    
    parsed_classes.append(row['class_id'])
    #print(parsed_classes[row[0]])

np_images = np.asarray(parsed_images) 
np_classes = np.asarray(parsed_classes)

print(f"Датасет изображений к сохранению: {np_images.shape}")
print(f"Датасет классов к сохранению: {np_classes.shape}")

idx.convert_to_file(DATASET_IDX_IMG, np_images)
idx.convert_to_file(DATASET_IDX_CLS, np_classes)




Общее количество: 100000


100000it [04:09, 400.77it/s]


Датасет изображений к сохранению: (100000, 28, 28)
Датасет классов к сохранению: (100000,)


In [17]:
images = idx.convert_from_file(DATASET_IDX_IMG)
classes = idx.convert_from_file(DATASET_IDX_CLS)

print(f"Датасет изображений из файла: {images.shape}")
print(f"Датасет классов из файла: {classes.shape}")

Датасет изображений из файла: (100000, 28, 28)
Датасет классов из файла: (100000,)


In [33]:
#images = idx.convert_from_file(DATASET_IDX_IMG)
#classes = idx.convert_from_file(r"D:\work\test_comp_vision\datasets\emnist_gzip\emnist-byclass-train-labels-idx1-ubyte")

#print(f"Датасет изображений из файла: {images.shape}")
#print(f"Датасет классов из файла: {classes.shape}")

Датасет классов из файла: (697932,)


## .H5 файл генератор (не используем, т.к. проще было пристроить .IDX)

In [ ]:
#Создаем файл датасета для обучения детектора
#train_dataset = DataGenerator(load_images_into_memory=False, hdf5_dataset_path=None)
#images_dir = ''

#train_labels_filename = EXPORT_NAME_CSV

#train_dataset.parse_csv(images_dir=images_dir,
#                        labels_filename=train_labels_filename,
#                        input_format=['image_name', 'xmin', 'xmax', 'ymin', 'ymax', 'class_id'], # This is the order of the first six columns in the CSV file that contains the labels for your dataset. If your labels are in XML format, maybe the XML parser will be helpful, check the documentation.
#                        #input_format=['image_name', 'class_id'],
#                        include_classes='all')

#train_dataset.create_hdf5_dataset(file_path=DATASET_H5,
#                                  resize=[IMAGE_HEIGHT, IMAGE_WIDTH],
#                                  variable_image_size=True,
#                                  verbose=True)